In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


MNIST_path = os.path.join(os.path.expanduser('~'), '.keras/datasets/mnist.npz')
ROOT_DIR = '/Users/halmagyi/Documents/MachineLearning/ML_Notes/BaysianNNets/BayesianNets'
# ROOT_DIR = '/home/ubuntu/Documents/BayesianNets'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
os.chdir(ROOT_DIR)
from src.mnist import *


In [2]:
num_classes = 10
x_train_flat, x_test_flat, x_train_flat_bias, x_test_flat_bias, Y_train, Y_test = make_mnist_data(num_classes)

num_train_samples, data_length = x_train_flat.shape

In [20]:
hidden_width = 300

hidden_activation = 'relu'
output_activation = 'softmax'

stddev = data_length ** (-1 / 2)
seed = 42
kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=stddev, seed=seed)

use_bias = True
dropout = False


################
# Regularizer
################
hidden_l2 = 0.00
output_l2=0.001
hidden_regularizer = tf.keras.regularizers.l2(hidden_l2)
output_kernel_regularizer = tf.keras.regularizers.l2(output_l2)


################
# Model
################

inputs = Input(shape=(data_length,))

x = Dense(hidden_width, activation=hidden_activation,
                  use_bias=use_bias,
                  kernel_initializer=kernel_initializer,
                  kernel_regularizer=hidden_regularizer)(inputs)

outputs = Dense(num_classes,
                    activation=output_activation,
                    kernel_initializer=kernel_initializer,
                    use_bias=use_bias,
                    kernel_regularizer=output_kernel_regularizer)(x)

model = Model(inputs=inputs, outputs=outputs)

optimizer = Adam()

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


batch_size = 32
epochs = 5

class EvaluateAfterBatch(Callback):
    """
    A custom callback class which will evaulate data after each batch has run.
    """
    def __init__(self, data):
        self.X, self.Y = data
        self.loss = []
        self.acc = []

    def on_batch_end(self, batch, logs={}):
#         loss, acc = self.model.evaluate(self.X, self.Y, verbose=0)
#         self.loss += [loss]
#         self.acc += [acc]
        pass

callbacks = [EvaluateAfterBatch((x_test_flat, Y_test))]



history = model.fit(x=x_train_flat, 
          y=Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          callbacks =callbacks,
          validation_data = (x_test_flat, Y_test))

Epoch 1/5
1875/1875 - 4s - loss: 0.2574 - accuracy: 0.9314 - val_loss: 0.1341 - val_accuracy: 0.9670 - 4s/epoch - 2ms/step
Epoch 2/5
1875/1875 - 3s - loss: 0.1246 - accuracy: 0.9699 - val_loss: 0.1118 - val_accuracy: 0.9733 - 3s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 3s - loss: 0.0938 - accuracy: 0.9794 - val_loss: 0.1013 - val_accuracy: 0.9758 - 3s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 3s - loss: 0.0770 - accuracy: 0.9841 - val_loss: 0.0942 - val_accuracy: 0.9775 - 3s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 3s - loss: 0.0667 - accuracy: 0.9864 - val_loss: 0.1007 - val_accuracy: 0.9758 - 3s/epoch - 2ms/step
